# Basic Imports

In [1]:
# File handling imports
import pickle
import argparse
import logging
import math
import os
import random
import time
from copy import deepcopy
from pathlib import Path
from threading import Thread

# Modelling imports
import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Helpers....
import test 
from models.experimental import attempt_load

# calling a new updated code for CLIP training
from models.yolo_zsd import Model

from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    fitness, strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, print_mutation, set_logging, one_cycle, colorstr
from utils.google_utils import attempt_download

# Loss function...
from utils.loss import ComputeLoss, ComputeLossOTA, ComputeLossOTA_CLIP, ComputeLoss_CLIP
from utils.plots import plot_images, plot_labels, plot_results, plot_evolution
from utils.torch_utils import ModelEMA, select_device, intersect_dicts, torch_distributed_zero_first, is_parallel
from utils.wandb_logging.wandb_utils import WandbLogger, check_wandb_resume

# Defining HyperParams for open world

In [2]:
# Setting Up cmd line arguments....
class agP():
    def __init__(self):
        self.weights = "./best_supervised_65.pt"
        self.cfg  = "/Users/vedant_j/Desktop/open_world/yolov7_open_world/cfg/training/yolo_v7_open_world.yaml"
        self.data = "/Users/vedant_j/Desktop/open_world/yolov7_open_world/data/open_world_65_split_yolo.yaml"
        self.hyp  = "/Users/vedant_j/Desktop/open_world/yolov7_open_world/data/hyp.scratch.p5.open_world.yaml"
        self.epochs = 50
        self.batch_size = 2
        self.img_size = [224, 224]
        self.rect = False
        self.resume = False
        self.nosave = False
        self.notest = False
        self.noautoanchor = False
        self.evolve = False
        self.cache_images = False
        self.image_weights = False
        self.device = "cpu"
        self.multi_scale = False
        self.single_cls = False
        self.adam = False
        self.sync_bn = False
        self.local_rank = -1
        self.workers = 8
        self.entity = None
        self.exist_ok = False
        self.quad = False
        self.linear_lr = False 
        self.label_smoothing = 0.0
        self.upload_dataset = False
        self.bbox_interval = -1
        self.save_period = -1
        self.artifact_alias = "latest"
        self.freeze = [0]
        self.v5_metric = False
        self.open_world = True
        self.open_world_emb_size = 512

In [3]:
opt = agP()
 # Set DDP variables (for distributed training)
opt.world_size = int(os.environ['WORLD_SIZE']) if 'WORLD_SIZE' in os.environ else 1 # only one node
opt.global_rank = int(os.environ['RANK']) if 'RANK' in os.environ else -1 # rank of the gpu in a single node
opt.total_batch_size = opt.batch_size

# Hyperparameters (using the base file)
with open(opt.hyp) as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader) 

# Train function

In [6]:
def train(hyp, opt, device):

    # extracting logs & hyper-param variables....
    epochs, batch_size, total_batch_size, weights, rank, freeze = opt.epochs, opt.batch_size, opt.total_batch_size, opt.weights, opt.global_rank, opt.freeze

    opt.hyp = hyp
    data_dict = None
    with open(opt.data) as f:
        data_dict = yaml.load(f, Loader=yaml.SafeLoader) 

    # number of classes
    nc = 1 if opt.single_cls else int(data_dict['nc'])  
    names = ['item'] if opt.single_cls and len(data_dict['names']) != 1 else data_dict['names']  # class names
    print("Number of classes : ",len(names))
    

    if opt.open_world:
        print("Updating number of classes from {} to {} for open world training.".format(nc, opt.open_world_emb_size))
        
        # updating nc
        nc = opt.open_world_emb_size

        # model check (ensuring open world model config file is being called....)
        assert "open_world" in opt.cfg 

        # loading class vectors from parent model....
        vector_pickle_paths = "/Users/vedant_j/Desktop/open_world/yolov7_open_world/className_embeddings.pickle"
        with open(vector_pickle_paths, 'rb') as handle:
            # the order in which classes.txt is created is the order in which elements of the text pickle are populated
            data_dict["vectors"] = pickle.load(handle)

            # mapping to be used to get the correct class vector...
            data_dict["number_to_class"] = {ctr : cName for ctr, cName in enumerate(data_dict["vectors"].keys())}
            class_vector_dict = {ctr : embVal for ctr, embVal in enumerate(data_dict["vectors"].values()) if ctr < data_dict['nc']}
        print("Loaded vectors count : ",len(class_vector_dict))
    
   
    # Model (PARTIAL WEIGHT LOADING FOR OPEN WORLD MODELS....)
    pretrained = weights.endswith('.pt')
   
    # fresh model instantiation......
    model = Model(opt.cfg, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create

    # Loading open world.....
    if opt.open_world:
        print("Fresh OPEN WORLD MODEL with base supervised weights....")
        # the open world trianing always starts from supervised pre-trained
        ckpt = torch.load(weights, map_location=device)
        exclude = ['anchor'] if (opt.cfg or hyp.get('anchors')) and not opt.resume else []  # exclude keys

        # LAYER 105 for corresponds to the object detection HEAD that we are trying to modify....
        exclude.extend(['model.105.anchors', 'model.105.anchor_grid', 'model.105.m.0.weight', 
                        'model.105.m.0.bias', 'model.105.m.1.weight', 'model.105.m.1.bias', 
                        'model.105.m.2.weight', 'model.105.m.2.bias'])
        
        # loading only the backbone weights + neck weights for supervised YOLO
        state_dict = ckpt['model'].float().state_dict()  # to FP32

        # Only loading weights for base & neck.....
        state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=exclude)  # intersect
        # loading open-world with SUPERVISED YOLO WEIGHTS
        model.load_state_dict(state_dict, strict=False) 



    # dataset path load ....
    train_path = data_dict['train']
    test_path = data_dict['val']
    print("Train path : ",train_path)
    print("Valid path : ",test_path)


    # Freeze [custom frozen layers ...]
    freeze = [f'model.{x}.' for x in (freeze if len(freeze) > 1 else range(freeze[0]))]  # parameter names to freeze (full or partial)
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers

        # freezing the named layers
        if any(x in k for x in freeze):
            print('freezing %s' % k)
            v.requires_grad = False

    # Training hyper-params definition 
    # Optimizer
    nbs = 64  # nominal batch size
    accumulate = max(round(nbs / total_batch_size), 1)  # accumulate loss before optimizing
    hyp['weight_decay'] *= total_batch_size * accumulate / nbs  # scale weight_decay
    

    pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
    for k, v in model.named_modules():
        if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):
            pg2.append(v.bias)  # biases
        if isinstance(v, nn.BatchNorm2d):
            pg0.append(v.weight)  # no decay
        elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):
            pg1.append(v.weight)  # apply decay
        if hasattr(v, 'im'):
            if hasattr(v.im, 'implicit'):           
                pg0.append(v.im.implicit)
            else:
                for iv in v.im:
                    pg0.append(iv.implicit)
        if hasattr(v, 'imc'):
            if hasattr(v.imc, 'implicit'):           
                pg0.append(v.imc.implicit)
            else:
                for iv in v.imc:
                    pg0.append(iv.implicit)
        if hasattr(v, 'imb'):
            if hasattr(v.imb, 'implicit'):           
                pg0.append(v.imb.implicit)
            else:
                for iv in v.imb:
                    pg0.append(iv.implicit)
        if hasattr(v, 'imo'):
            if hasattr(v.imo, 'implicit'):           
                pg0.append(v.imo.implicit)
            else:
                for iv in v.imo:
                    pg0.append(iv.implicit)
        if hasattr(v, 'ia'):
            if hasattr(v.ia, 'implicit'):           
                pg0.append(v.ia.implicit)
            else:
                for iv in v.ia:
                    pg0.append(iv.implicit)
        if hasattr(v, 'attn'):
            if hasattr(v.attn, 'logit_scale'):   
                pg0.append(v.attn.logit_scale)
            if hasattr(v.attn, 'q_bias'):   
                pg0.append(v.attn.q_bias)
            if hasattr(v.attn, 'v_bias'):  
                pg0.append(v.attn.v_bias)
            if hasattr(v.attn, 'relative_position_bias_table'):  
                pg0.append(v.attn.relative_position_bias_table)
        if hasattr(v, 'rbr_dense'):
            if hasattr(v.rbr_dense, 'weight_rbr_origin'):  
                pg0.append(v.rbr_dense.weight_rbr_origin)
            if hasattr(v.rbr_dense, 'weight_rbr_avg_conv'): 
                pg0.append(v.rbr_dense.weight_rbr_avg_conv)
            if hasattr(v.rbr_dense, 'weight_rbr_pfir_conv'):  
                pg0.append(v.rbr_dense.weight_rbr_pfir_conv)
            if hasattr(v.rbr_dense, 'weight_rbr_1x1_kxk_idconv1'): 
                pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_idconv1)
            if hasattr(v.rbr_dense, 'weight_rbr_1x1_kxk_conv2'):   
                pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_conv2)
            if hasattr(v.rbr_dense, 'weight_rbr_gconv_dw'):   
                pg0.append(v.rbr_dense.weight_rbr_gconv_dw)
            if hasattr(v.rbr_dense, 'weight_rbr_gconv_pw'):   
                pg0.append(v.rbr_dense.weight_rbr_gconv_pw)
            if hasattr(v.rbr_dense, 'vector'):   
                pg0.append(v.rbr_dense.vector)

    if opt.adam:
        optimizer = optim.Adam(pg0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
    else:
        optimizer = optim.SGD(pg0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)

    optimizer.add_param_group({'params': pg1, 'weight_decay': hyp['weight_decay']})  # add pg1 with weight_decay
    optimizer.add_param_group({'params': pg2})  # add pg2 (biases)
    del pg0, pg1, pg2

    # Scheduler https://arxiv.org/pdf/1812.01187.pdf
    # https://pytorch.org/docs/stable/_modules/torch/optim/lr_scheduler.html#OneCycleLR
    if opt.linear_lr:
        lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - hyp['lrf']) + hyp['lrf']  # linear
    else:
        lf = one_cycle(1, hyp['lrf'], epochs)  # cosine 1->hyp['lrf']
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)
    

    # EMA
    ema = ModelEMA(model) if rank in [-1, 0] else None

    # Resume
    start_epoch, best_fitness = 0, 0.0
    
    # Image sizes
    gs = max(int(model.stride.max()), 32)  # grid size (max stride)
    nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
    imgsz, imgsz_test = [check_img_size(x, gs) for x in opt.img_size]  # verify imgsz are gs-multiples

    # DP mode
    cuda = False
    if cuda and rank == -1 and torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    # SyncBatchNorm (in distributed training this is a very important flag to set)
    if opt.sync_bn and cuda and rank != -1:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)


    # keeping augmentations off for open world training.... (first phase)
    dataloader, dataset = create_dataloader(test_path, 
                                            imgsz, 
                                            batch_size, 
                                            gs, 
                                            opt,
                                            hyp=hyp, 
                                            augment=False, 
                                            cache=opt.cache_images, 
                                            rect=opt.rect, 
                                            rank=rank,
                                            world_size=opt.world_size, 
                                            workers=opt.workers,
                                            image_weights=opt.image_weights, 
                                            quad=opt.quad, 
                                            prefix=colorstr('train: '))


    
    nb = len(dataloader)  # number of batches
    print("Number of batches :",nb)

    

    # DDP mode (incase rank is speicified)
    if cuda and rank != -1:
        model = DDP(model, device_ids=[opt.local_rank], output_device=opt.local_rank,
                    # nn.MultiheadAttention incompatibility with DDP https://github.com/pytorch/pytorch/issues/26698
                    find_unused_parameters=any(isinstance(layer, nn.MultiheadAttention) for layer in model.modules()))

    # Model parameters

    # yolo loss parameters.....
    hyp['box'] *= 3. / nl  # scale to layers
    hyp['cls'] *= nc / 80. * 3. / nl  # scale to classes and layers
    hyp['obj'] *= (imgsz / 640) ** 2 * 3. / nl  # scale to image size and layers
    hyp['label_smoothing'] = opt.label_smoothing
    model.nc = nc  # attach number of classes to model
    model.hyp = hyp  # attach hyperparameters to model
    model.gr = 1.0  # iou loss ratio (obj_loss = 1.0 or iou)
    if not opt.open_world:
        model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
    else:
        # for open  world case classes don't exist per se....
        model.class_weights = None
    
    model.names = names


    # Start training
    t0 = time.time()
    nw = max(round(hyp['warmup_epochs'] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
    # nw = min(nw, (epochs - start_epoch) / 2 * nb)  # limit warmup to < 1/2 of training
    maps = np.zeros(nc)  # mAP per class
    results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
    scheduler.last_epoch = start_epoch - 1  # do not move
    scaler = amp.GradScaler(enabled=cuda)
    
    # initialzing loss classes....
    if not opt.open_world:
        compute_loss_ota = ComputeLossOTA(model)  # init loss class
        compute_loss = ComputeLoss(model)  # init loss class

    else:
        compute_loss_ota = ComputeLossOTA_CLIP(model, clip_text_vectors = class_vector_dict)
        compute_loss = ComputeLoss_CLIP(model, clip_text_vectors = class_vector_dict)  # init loss class


    print("Starting to train")


    # Initiating training sequence .....
    for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
        # This operation sets the flag : model.training
        model.train()

        # Not focusing on this for open world training....
        # Update image weights (optional)

        mloss = torch.zeros(4, device=device)  # mean losses
        if rank != -1:
            dataloader.sampler.set_epoch(epoch)
            
        pbar = enumerate(dataloader)
        if rank in [-1, 0]:
            pbar = tqdm(pbar, total=nb)  # progress bar

        # intializing back prop...
        optimizer.zero_grad()

        # loading each batch.....
        for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
            ni = i + nb * epoch  # number integrated batches (since train start) [total count of seen batches]
            imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

            # Warmup (warmup to avoid getting stuck in local optima on the loss surface)
            if ni <= nw:
                xi = [0, nw]  # x interp
                # model.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
                accumulate = max(1, np.interp(ni, xi, [1, nbs / total_batch_size]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

            # Multi-scale (opt is from read cmd line args)
            if opt.multi_scale:
                sz = random.randrange(imgsz * 0.5, imgsz * 1.5 + gs) // gs * gs  # size
                sf = sz / max(imgs.shape[2:])  # scale factor
                if sf != 1:
                    ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
                    imgs = F.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)

            # Forward (amp : mixed precision training)
            with amp.autocast(enabled=cuda):
                pred = model(imgs)  # forward pass

                # loss computation (embeddings will be loaded on GPU)
                if 'loss_ota' not in hyp or hyp['loss_ota'] == 1:
                    loss, loss_items = compute_loss_ota(pred, 
                                                        targets.to(device), 
                                                        imgs)  # loss scaled by batch_size
                else:
                    loss, loss_items = compute_loss(pred, 
                                                    targets.to(device))  # loss scaled by batch_size
                if rank != -1:
                    loss *= opt.world_size  # gradient averaged between devices in DDP mode
                if opt.quad:
                    loss *= 4.

            # Backward
            scaler.scale(loss).backward()

            # Optimize
            if ni % accumulate == 0:
                scaler.step(optimizer)  # optimizer.step
                scaler.update()
                optimizer.zero_grad()
                if ema:
                    ema.update(model)

            # Print
            if rank in [-1, 0]:
                mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
                mem = '%.3gG' % (torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0)  # (GB)
                s = ('%10s' * 2 + '%10.4g' * 6) % (
                    '%g/%g' % (epoch, epochs - 1), mem, *mloss, targets.shape[0], imgs.shape[-1])
                pbar.set_description(s)


            # end batch ------------------------------------------------------------------------------------------------
        # end epoch ----------------------------------------------------------------------------------------------------

        # Scheduler
        lr = [x['lr'] for x in optimizer.param_groups]  # for tensorboard
        scheduler.step()

        # DDP process 0 or single-GPU
        if rank in [-1, 0]:
            # mAP
            ema.update_attr(model, include=['yaml', 'nc', 'hyp', 'gr', 'names', 'stride', 'class_weights'])
            final_epoch = epoch + 1 == epochs
            if not opt.notest or final_epoch:  # Calculate mAP
                wandb_logger.current_epoch = epoch + 1
                results, maps, times = test.test(data_dict,
                                                 batch_size=batch_size * 2,
                                                 imgsz=imgsz_test,
                                                 model=ema.ema,
                                                 single_cls=opt.single_cls,
                                                 dataloader=testloader,
                                                 save_dir=save_dir,
                                                 verbose=nc < 50 and final_epoch,
                                                 plots=plots and final_epoch,
                                                 wandb_logger=wandb_logger,
                                                 compute_loss=compute_loss,
                                                 is_coco=is_coco,
                                                 v5_metric=opt.v5_metric)

            # Write
            with open(results_file, 'a') as f:
                f.write(s + '%10.4g' * 7 % results + '\n')  # append metrics, val_loss
            if len(opt.name) and opt.bucket:
                os.system('gsutil cp %s gs://%s/results/results%s.txt' % (results_file, opt.bucket, opt.name))

            # Log
            tags = ['train/box_loss', 'train/obj_loss', 'train/cls_loss',  # train loss
                    'metrics/precision', 'metrics/recall', 'metrics/mAP_0.5', 'metrics/mAP_0.5:0.95',
                    'val/box_loss', 'val/obj_loss', 'val/cls_loss',  # val loss
                    'x/lr0', 'x/lr1', 'x/lr2']  # params
            for x, tag in zip(list(mloss[:-1]) + list(results) + lr, tags):
                if tb_writer:
                    tb_writer.add_scalar(tag, x, epoch)  # tensorboard
                if wandb_logger.wandb:
                    wandb_logger.log({tag: x})  # W&B

            # Update best mAP
            fi = fitness(np.array(results).reshape(1, -1))  # weighted combination of [P, R, mAP@.5, mAP@.5-.95]
            if fi > best_fitness:
                best_fitness = fi
            wandb_logger.end_epoch(best_result=best_fitness == fi)

            # Save model
            if (not opt.nosave) or (final_epoch and not opt.evolve):  # if save
                ckpt = {'epoch': epoch,
                        'best_fitness': best_fitness,
                        'training_results': results_file.read_text(),
                        'model': deepcopy(model.module if is_parallel(model) else model).half(),
                        'ema': deepcopy(ema.ema).half(),
                        'updates': ema.updates,
                        'optimizer': optimizer.state_dict(),
                        'wandb_id': wandb_logger.wandb_run.id if wandb_logger.wandb else None}

                # Save last, best and delete
                torch.save(ckpt, last)
                if best_fitness == fi:
                    torch.save(ckpt, best)
                if (best_fitness == fi) and (epoch >= 200):
                    torch.save(ckpt, wdir / 'best_{:03d}.pt'.format(epoch))
                if epoch == 0:
                    torch.save(ckpt, wdir / 'epoch_{:03d}.pt'.format(epoch))
                elif ((epoch+1) % 25) == 0:
                    torch.save(ckpt, wdir / 'epoch_{:03d}.pt'.format(epoch))
                elif epoch >= (epochs-5):
                    torch.save(ckpt, wdir / 'epoch_{:03d}.pt'.format(epoch))
                if wandb_logger.wandb:
                    if ((epoch + 1) % opt.save_period == 0 and not final_epoch) and opt.save_period != -1:
                        wandb_logger.log_model(
                            last.parent, opt, epoch, fi, best_model=best_fitness == fi)
                del ckpt

        # end epoch ----------------------------------------------------------------------------------------------------
    # end training
    if rank in [-1, 0]:
        # Plots
        if plots:
            plot_results(save_dir=save_dir)  # save as results.png
            if wandb_logger.wandb:
                files = ['results.png', 'confusion_matrix.png', *[f'{x}_curve.png' for x in ('F1', 'PR', 'P', 'R')]]
                wandb_logger.log({"Results": [wandb_logger.wandb.Image(str(save_dir / f), caption=f) for f in files
                                              if (save_dir / f).exists()]})
        # Test best.pt
        logger.info('%g epochs completed in %.3f hours.\n' % (epoch - start_epoch + 1, (time.time() - t0) / 3600))
        if opt.data.endswith('coco.yaml') and nc == 80:  # if COCO
            for m in (last, best) if best.exists() else (last):  # speed, mAP tests
                results, _, _ = test.test(opt.data,
                                          batch_size=batch_size * 2,
                                          imgsz=imgsz_test,
                                          conf_thres=0.001,
                                          iou_thres=0.7,
                                          model=attempt_load(m, device).half(),
                                          single_cls=opt.single_cls,
                                          dataloader=testloader,
                                          save_dir=save_dir,
                                          save_json=True,
                                          plots=False,
                                          is_coco=is_coco,
                                          v5_metric=opt.v5_metric)

        # Strip optimizers
        final = best if best.exists() else last  # final model
        for f in last, best:
            if f.exists():
                strip_optimizer(f)  # strip optimizers
        if opt.bucket:
            os.system(f'gsutil cp {final} gs://{opt.bucket}/weights')  # upload
        if wandb_logger.wandb and not opt.evolve:  # Log the stripped model
            wandb_logger.wandb.log_artifact(str(final), type='model',
                                            name='run_' + wandb_logger.wandb_run.id + '_model',
                                            aliases=['last', 'best', 'stripped'])
        wandb_logger.finish_run()
    else:
        dist.destroy_process_group()
    torch.cuda.empty_cache()
    return results


In [7]:
train(hyp, opt, device = torch.device('cpu'))

Number of classes :  65
Updating number of classes from 65 to 512 for open world training.
Loaded vectors count :  65
Fresh OPEN WORLD MODEL with base supervised weights....
Train path :  /Users/vedant_j/Desktop/open_world/yolo_coco_dataset_65_15/train/images
Valid path :  /Users/vedant_j/Desktop/open_world/yolo_coco_dataset_65_15/val_yolo/images


train: Scanning '/Users/vedant_j/Desktop/open_world/yolo_coco_dataset_65_15/val_yolo/label_clip_vecto

train: WARNING: Ignoring corrupted image and/or label /Users/vedant_j/Desktop/open_world/yolo_coco_dataset_65_15/val_yolo/images/COCO_train2014_000000214087.jpg: duplicate labels


train: Scanning '/Users/vedant_j/Desktop/open_world/yolo_coco_dataset_65_15/val_yolo/label_clip_vecto


16343
Number of batches : 8172
Loaded CLIP text classification vectors :  torch.Size([65, 512])
Starting to train


      0/49        0G   0.07504  0.002552     478.1     478.2        26       224:   1%| | 93/8172 [00


KeyboardInterrupt: 

## Test BED

In [ ]:
for coup in dataloader:
    break

In [ ]:
img, labels_out, paths, shapes = coup

In [ ]:
from PIL import Image
import matplotlib.patches as patches

In [ ]:
xmin = []
ymin = []
w_list = []
h_list = []
labs_list = []
cnt = 10
im = Image.fromarray(img[cnt].permute(1, 2, 0).numpy())


for i in labels_out:
    if i[0] == cnt:
        x_c = (i[2]*640).numpy()
        y_c = (i[3]*640).numpy()
        w = (i[4]*640).numpy()
        h = (i[5]*640).numpy()
        xmin.append(x_c - w/2)
        ymin.append(y_c - h/2)
        w_list.append(w)
        h_list.append(h)
        labs_list.append(i[1])


# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

for x_i,y_i,w_i,h_i in zip(xmin,ymin,w_list,h_list):

    # Create a Rectangle patch
    rect = patches.Rectangle((x_i, y_i), w_i, h_i, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()

In [ ]:
labs_list

In [ ]:
vector_pickle_paths = "/Users/vedant_j/Desktop/open_world/yolov7_open_world/className_embeddings.pickle"
data_dict = {'nc' : 65}
with open(vector_pickle_paths, 'rb') as handle:
    # the order in which classes.txt is created is the order in which elements of the text pickle are populated
    data_dict["vectors"] = pickle.load(handle)

    # mapping to be used to get the correct class vector...
    data_dict["number_to_class"] = {ctr : cName for ctr, cName in enumerate(data_dict["vectors"].keys())}
    class_vector_dict = {ctr : embVal for ctr, embVal in enumerate(data_dict["vectors"].values()) if ctr < data_dict['nc']}
print("Loaded vectors count : ",len(class_vector_dict))

In [ ]:
gain = torch.ones(7, device=torch.device('cpu')).long() 
targets = torch.from_numpy(np.array([[[1.0,2.0,0.4,0.6,0.7,0.8],
                                     [2.0,2.0,0.3,0.6,0.4,0.8]]]))

In [ ]:
targets*gain

In [ ]:
t1 = torch.from_numpy(np.array([[1,3,4,5,6],
                                [4,3,4,5,2],
                                [5,3,4,5,4],
                                [6,7,7,7,6]], dtype=np.float32))


t2 = torch.from_numpy(np.array([[2,8,10,11,1],
                                [4,3,4,5,7],
                                [6,7,4,1,2]], dtype=np.float32))

In [ ]:
store_vec  = torch.zeros((t1.shape[0], t2.shape[0])).shape

In [ ]:
a_norm = t1 / t1.norm(dim=1)[:, None]
b_norm = t2 / t2.norm(dim=1)[:, None]
res = torch.mm(a_norm, b_norm.transpose(0,1))
res.shape

In [ ]:
res

In [ ]:
target = torch.from_numpy(np.array([0,2,2,1]))
target.shape

In [ ]:
torch.nn.NLLLoss(reduction="mean")(res.log(), target)

In [ ]:
res.log()

In [ ]:
np.log(0.95976)

In [8]:
s = "/train/images"

In [10]:
s[:-6]

'/train/'